# Customer Churn — Data Cleaning

## Objective
This notebook cleans the raw Telco Customer Churn dataset based on insights from
`01_data_understanding.ipynb` and rules defined in `schema.yaml`.

We will:
- Handle blank values
- Fix data type issues
- Remove identifier column
- Validate final schema
- Prepare clean dataset for transformation & modeling




In [12]:
import pandas as pd
import numpy as np

In [13]:
pd.set_option("display.max_columns", None)

In [14]:
DATA_PATH = "../data/raw/Telco-Customer-Churn.csv"

df = pd.read_csv(DATA_PATH)

print("Initial Shape:", df.shape)
df.head()


Initial Shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Known Issues from Data Understanding

1. `TotalCharges` stored as object
2. Blank values present in `TotalCharges` (11 rows)
3. `customerID` is an identifier (not a feature)
4. No duplicate rows


In [15]:
# convert blank string to Nan
df.replace(" ", np.nan, inplace=True)

df.isna().sum()


customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [16]:
## Handle missing values in TotalCharges
df[df["TotalCharges"].isna()][["tenure", "MonthlyCharges", "TotalCharges"]]


,tenure,MonthlyCharges,TotalCharges
488,0,52.55,NaN
753,0,20.25,NaN
936,0,80.85,NaN
1082,0,25.75,NaN
1340,0,56.05,NaN
3331,0,19.85,NaN
3826,0,25.35,NaN
4380,0,20.00,NaN
5218,0,19.70,NaN
6670,0,73.35,NaN


In [17]:
df = df.dropna(subset=["TotalCharges"])

print("Shape after removing blanks:", df.shape)


Shape after removing blanks: (7032, 21)


In [18]:
# Convert TotalCharges to Numeric
df["TotalCharges"] = df["TotalCharges"].astype(float)

df.dtypes


customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [19]:
# Drop Identifier column
df.drop(columns=["customerID"], inplace=True)

print("Shape after dropping ID:", df.shape)


Shape after dropping ID: (7032, 20)


In [ ]:
# Validate target columns
df["Churn"].value_counts()


Churn
No     5163
Yes    1869
Name: count, dtype: int64

In [23]:
df["Churn"].value_counts(normalize=True)


Churn
No     0.734215
Yes    0.265785
Name: proportion, dtype: float64

In [ ]:
# Validate the Data Types
categorical_cols = df.select_dtypes(include="object").columns
numerical_cols = df.select_dtypes(exclude="object").columns

print("Categorical Columns:", list(categorical_cols))
print("Numerical Columns:", list(numerical_cols))


Categorical Columns: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']
Numerical Columns: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']


In [26]:
df.isnull().sum().sum()

np.int64(0)

In [27]:
df.shape

(7032, 20)

## Final Clean Dataset Summary

- Rows: 7032
- Columns: 20
- Missing values: ❌ None
- Duplicates: ❌ None
- Data types: ✅ Correct
- Identifier column: ❌ Removed

Dataset is now ready for:
➡️ Feature transformation & encoding


In [30]:
CLEAN_PATH = "../data/processed/cleaned_telco_churn.csv"

df.to_csv(CLEAN_PATH, index=False)

print(f"Clean dataset saved to: {CLEAN_PATH}")


Clean dataset saved to: ../data/processed/cleaned_telco_churn.csv
